In [10]:
from os import listdir
from os.path import isfile
from json import load
from shutil import rmtree
from shutil import copytree
from os import makedirs

In [11]:
PASTA = "requisicoes"    
ORDEM_DA_LUZ = 'ordem_da_luz'

In [12]:
total_respondidas = 0
total_sem_responder  = 0
duracoes = []
for item in listdir(PASTA):
    caminho = f'{PASTA}/{item}'
    caminho_resposta = f'{caminho}/resposta.json'
 
    if not isfile(caminho_resposta):
        print("sem responder em: ", item)
        total_sem_responder+=1
        continue

    total_respondidas+=1
    with open(caminho_resposta,'r') as arq:
        resposta = load(arq)
    
    inicio = resposta['inico']
    fim = resposta['fim']
    duracao = fim - inicio
    duracoes.append(duracao)

print('duracao mínima',min(duracao))
print('duracao máxima',max(duracao))
print('media',sum(duracao) /len(duracao))
print("total: ", total_respondidas + total_sem_responder)
print("total respondidas: ",total_respondidas)
print("total sem responder",total_sem_responder)


total:  58255
total respondidas:  58255
total sem responder 0


In [13]:
valores_da_ordem_da_luz = []

In [14]:
def modifica_num(valor,quantidade_max):
    tamanho = len(str(valor))
    total_0 = quantidade_max - tamanho
    final = ''
    for i in range(0,total_0):
        final+='0'
    final+=str(valor)
    return final



Organizando a ordem


In [15]:
def gera_ordem_da_luz():
    rmtree(ORDEM_DA_LUZ,ignore_errors=True)
    makedirs(ORDEM_DA_LUZ)

    listagem = listdir(PASTA)
    for item in listagem:
        caminho = f'{PASTA}/{item}'
        caminho_resposta = f'{caminho}/resposta.json'
        if not isfile(caminho_resposta):
            continue

        with open(caminho_resposta,'r') as arq:
            respostas = load(arq)



        caminho_request = f'{caminho}/request.json'
        with open(caminho_request,'r') as arq:
            request = load(arq)
            
        rota:str = request['rota']
        rota_dividida = rota.split('/')
        id_cliente = rota_dividida[2]

        start_segundos = modifica_num(respostas["luz_adiquirida"]["segundos"],10)
        start_nano = modifica_num(respostas["luz_adiquirida"]["nanosegundos"],10)

        start = f'{start_segundos}:{start_nano}'

        end = ''
        if respostas.get('luz_liberada'):
            end_segundos = modifica_num(respostas["luz_liberada"]["segundos"],10)
            end_nano = modifica_num(respostas["luz_liberada"]["nanosegundos"],10)
            end = f'{end_segundos}:{end_nano}'
            
        novo_caminho =  f'{ORDEM_DA_LUZ}/cliente:{id_cliente}:{start}=>{end}|{item}'
        copytree(caminho,novo_caminho)
        valores_da_ordem_da_luz.append(item)
            



In [16]:
clientes = {
    "1":{
        "saldo":0,
        "limite":100000,
    },
    "2":{
        "saldo":0,
        "limite":80000
    },
    "3":{
        "saldo":0,
        "limite":1000000
    },
    "4":{
        "saldo":0,
        "limite":10000000
    },
    "5":{
        "saldo":0,
        "limite":500000
    }
}

In [17]:
def plotar_ordem_da_luz():
    ids = clientes.keys()
    listagem = listdir(ORDEM_DA_LUZ)
    listagem.sort()

    for item in listagem:

        caminho = f'{ORDEM_DA_LUZ}/{item}'
        caminho_request = f'{caminho}/request.json'
        with open(caminho_request,'r') as arq:
            request = load(arq)
        
        caminho_resposta = f'{caminho}/resposta.json'
        if not isfile(caminho_resposta):
            raise Exception('aaaaaaa')
        
        rota:str = request['rota']

        rota_dividida = rota.split('/')
        id_cliente = rota_dividida[2]
        if id_cliente not in ids:
            continue
        
        cliente_atual = clientes[id_cliente]
        
        tipo_rota = rota_dividida[3]
        if tipo_rota != 'transacoes':
            continue

        json_body = request['json_body']

        if json_body['valor'] < 0:
            continue
        
        if json_body['valor'].__class__ != int:
            continue

        if  not json_body.get('descricao'):
            continue

        if len(json_body['descricao']) == 0 or len(json_body['descricao'])> 10:
            continue


        if json_body['tipo'] == 'c':
            cliente_atual['saldo'] += json_body['valor']

        
        if json_body['tipo'] == 'd':
            saldo_full = cliente_atual['saldo']  + cliente_atual['limite']
            saldo_full-=json_body['valor']
            if saldo_full < 0:
                continue

            cliente_atual['saldo'] -= json_body['valor']
       
        caminho_resposta = f'{caminho}/resposta.json'
        
        with open(caminho_resposta,'r') as arq:
            resposta = load(arq)
   
        if not resposta.get('json_body'):
            raise Exception("aaaaaaaaaaa")
        
       
        #print(
        #    f'item: {item} =>' +
        #    f'cliente: {id_cliente} => '+
        #    f'saldo =>{cliente_atual["saldo"]}'
        #    )
     

In [18]:
clientes

{'1': {'saldo': 0, 'limite': 100000},
 '2': {'saldo': 0, 'limite': 80000},
 '3': {'saldo': 0, 'limite': 1000000},
 '4': {'saldo': 0, 'limite': 10000000},
 '5': {'saldo': 0, 'limite': 500000}}